In [16]:
# Step 8

# 📦 Load packages
library(dplyr)
library(tidyr)
library(lme4)
library(lmerTest)
library(ggplot2)

# ✅ Reshape data: Wide → Long
long_data <- clean_data %>%
  select(id, sepsis, atg_48h = atg_48_hours_48h_before_febrile_episode,
         presepsin_d1_ng_m_l, presepsin_d3_ng_m_l, presepsin_d5_ng_m_l, presepsin_d7_ng_m_l,
         ykl_40_d1_ng_m_l, ykl_40_d3_ng_m_l, ykl_40_d5_ng_m_l, ykl_40_d7_ng_m_l) %>%
  pivot_longer(cols = starts_with("presepsin") | starts_with("ykl_40"),
               names_to = c("biomarker", "day"),
               names_pattern = "(.*)_d(\\d+)_ng_m_l",
               values_to = "value") %>%
  mutate(
    day = as.factor(day),
    sepsis = factor(sepsis, levels = c("No Sepsis", "Sepsis")),
    atg_48h = factor(atg_48h, levels = c(0, 1), labels = c("No ATG", "Yes ATG"))
  ) %>%
  filter(!is.na(value))

# ✅ Split for each biomarker
psp_data <- long_data %>% filter(biomarker == "presepsin")
ykl_data <- long_data %>% filter(biomarker == "ykl_40")

# 🔁 Fit LMM for Presepsin
model_psp <- lmer(value ~ day + sepsis + atg_48h + (1 | id), data = psp_data)
summary(model_psp)

# 🔁 Fit LMM for YKL-40
model_ykl <- lmer(value ~ day + sepsis + atg_48h + (1 | id), data = ykl_data)
summary(model_ykl)


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack



Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step




Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: value ~ day + sepsis + atg_48h + (1 | id)
   Data: psp_data

REML criterion at convergence: 2085.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.6105 -0.2952 -0.0317  0.2266  8.6568 

Random effects:
 Groups   Name        Variance Std.Dev.
 id       (Intercept) 708.86   26.62   
 Residual              69.38    8.33   
Number of obs: 264, groups:  id, 66

Fixed effects:
               Estimate Std. Error      df t value Pr(>|t|)    
(Intercept)       4.107      4.308  68.669   0.953   0.3438    
day3             -2.082      1.450 195.000  -1.436   0.1526    
day5             -3.534      1.450 195.000  -2.437   0.0157 *  
day7             -5.972      1.450 195.000  -4.119 5.63e-05 ***
sepsisSepsis     14.283      8.263  63.000   1.729   0.0888 .  
atg_48hYes ATG   11.292      7.806  63.000   1.447   0.1530    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: value ~ day + sepsis + atg_48h + (1 | id)
   Data: ykl_data

REML criterion at convergence: 2862

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.3544 -0.4235 -0.0829  0.3614  3.8874 

Random effects:
 Groups   Name        Variance Std.Dev.
 id       (Intercept) 6261     79.13   
 Residual             1810     42.55   
Number of obs: 264, groups:  id, 66

Fixed effects:
               Estimate Std. Error      df t value Pr(>|t|)    
(Intercept)     100.009     13.598  79.358   7.355 1.52e-10 ***
day3             -5.678      7.406 195.000  -0.767 0.444183    
day5            -26.277      7.406 195.000  -3.548 0.000486 ***
day7            -18.997      7.406 195.000  -2.565 0.011070 *  
sepsisSepsis     22.637     25.124  63.000   0.901 0.371023    
atg_48hYes ATG   14.566     23.736  63.000   0.614 0.541653    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

C

Step 8: Repeated Measures Modeling (Linear Mixed-Effects Models) 

Objective
To evaluate the longitudinal behavior of Presepsin and YKL-40 biomarkers over Days 1, 3, 5, and 7 while controlling for:

Sepsis status

ATG exposure 48h prior to fever onset

Repeated measurements within individuals

Linear mixed-effects models (LMM) were fitted for each biomarker using day, sepsis, and ATG status as fixed effects and patient ID (id) as a random effect. 

Model Summary: Presepsin
Predictor	Estimate	p-value	Interpretation
Intercept (Day 1)	4.11	0.344	Baseline Presepsin level
Day 3	-2.08	0.153	↓ from Day 1, not statistically significant
Day 5	-3.53	0.016	Significant ↓ from Day 1
Day 7	-5.97	<0.001	Strong significant ↓ from Day 1
Sepsis	+14.28	0.089	Marginal ↑ in sepsis patients
ATG 48h	+11.29	0.153	↑ in patients with recent ATG, not significant 

Interpretation:

Presepsin levels decline significantly over time, especially by Day 5 and Day 7.

Sepsis has a marginal association with higher Presepsin (p = 0.089), suggesting a trend.

ATG exposure showed no significant independent effect after adjusting for other variables. 

Model Summary: YKL-40
Predictor	Estimate	p-value	Interpretation
Intercept (Day 1)	100.01	<0.001	Baseline YKL-40 level
Day 3	-5.68	0.444	↓ from Day 1, not significant
Day 5	-26.28	<0.001	Significant ↓ from Day 1
Day 7	-19.00	0.011	Significant ↓ from Day 1
Sepsis	+22.64	0.371	↑ in sepsis patients, not significant
ATG 48h	+14.57	0.542	↑ with ATG, not significant 

Interpretation:

YKL-40 levels also significantly decreased by Day 5 and Day 7.

Neither Sepsis nor ATG reached statistical significance, although both showed upward trends. 

Key Takeaways
Both biomarkers exhibit a decreasing pattern over time, especially from Day 5 onward.

After adjusting for within-subject variability:

Day remains a significant predictor of biomarker dynamics.

Sepsis shows a trend but is not consistently significant in explaining variation.

ATG (48h) effect is not statistically significant when modeled alongside day and sepsis. 


These findings suggest that temporal dynamics are crucial in interpreting biomarker values in post-transplant patients. While Presepsin may have stronger diagnostic sensitivity to sepsis (p ~ 0.089), repeated measures modeling confirms that Day of measurement is the most consistent predictor of biomarker changes.